<a href="https://colab.research.google.com/github/hbgit/app_spam_classification/blob/main/SpamClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 577 kB 41.1 MB/s 
     |████████████████████████████████| 3.4 MB 50.7 MB/s 
     |████████████████████████████████| 128 kB 89.1 MB/s 
     |████████████████████████████████| 1.1 MB 70.0 MB/s 
     |████████████████████████████████| 10.9 MB 51.8 MB/s 
     |████████████████████████████████| 238 kB 71.0 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 1.3 MB 73.3 MB/s 
     |████████████████████████████████| 60.2 MB 1.2 MB/s 
     |████████████████████████████████| 840 kB 95.9 MB/s 
     |████████████████████████████████| 25.3 MB 87.3 MB/s 
     |████████████████████████████████| 352 kB 65.6 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 40 kB 6.7 MB/s 
     |████████████████████████████████| 213 kB 71.9 MB/s 
     |████████████████████████████████| 1.1 MB 89.6 MB/s 


In [2]:
import numpy as np
import os
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')


In [3]:
data_file = tf.keras.utils.get_file(fname='comment-spam.csv', 
  origin='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/lmblog_comments.csv', 
  extract=False)

245760/235543 [===============================] - 0s 0us/step


In [4]:
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

In [5]:
data = DataLoader.from_csv(
    filename=data_file,
    text_column='commenttext',
    label_column='spam',
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True)

train_data, test_data = data.split(0.9)

In [6]:
# Build the model
model = text_classifier.create(train_data, model_spec=spec, epochs=50, 
                               validation_data=test_data)

28/28 [==============================] - 1s 31ms/step - loss: 0.6360 - accuracy: 0.7712 - val_loss: 0.5952 - val_accuracy: 0.7200
Epoch 2/2
28/28 [==============================] - 0s 5ms/step - loss: 0.5552 - accuracy: 0.7690 - val_loss: 0.5221 - val_accuracy: 0.7900
Epoch 3/3
28/28 [==============================] - 0s 5ms/step - loss: 0.4834 - accuracy: 0.8393 - val_loss: 0.4530 - val_accuracy: 0.8400
Epoch 4/4
28/28 [==============================] - 0s 5ms/step - loss: 0.4135 - accuracy: 0.8906 - val_loss: 0.3901 - val_accuracy: 0.8800
Epoch 5/5
28/28 [==============================] - 0s 6ms/step - loss: 0.3555 - accuracy: 0.9342 - val_loss: 0.3359 - val_accuracy: 0.9300
Epoch 6/6
28/28 [==============================] - 0s 5ms/step - loss: 0.3065 - accuracy: 0.9386 - val_loss: 0.2906 - val_accuracy: 0.9400
Epoch 7/7
28/28 [==============================] - 0s 5ms/step - loss: 0.2659 - accuracy: 0.9408 - val_loss: 0.2541 - val_accuracy: 0.9400
Epoch 8/8
28/28 [===================

In [8]:
!ls

sample_data


In [9]:
!mkdir mm_spam_savedmodel

In [11]:
model.export(export_dir='/mm_spam_savedmodel', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB, ExportFormat.SAVED_MODEL])

In [12]:
# Rename the SavedModel subfolder to a version number
!mv /mm_spam_savedmodel/saved_model /mm_spam_savedmodel/1
!zip -r mm_spam_savedmodel.zip /mm_spam_savedmodel/

  adding: mm_spam_savedmodel/ (stored 0%)
  adding: mm_spam_savedmodel/labels.txt (stored 0%)
  adding: mm_spam_savedmodel/1/ (stored 0%)
  adding: mm_spam_savedmodel/1/keras_metadata.pb (deflated 86%)
  adding: mm_spam_savedmodel/1/variables/ (stored 0%)
  adding: mm_spam_savedmodel/1/variables/variables.data-00000-of-00001 (deflated 35%)
  adding: mm_spam_savedmodel/1/variables/variables.index (deflated 59%)
  adding: mm_spam_savedmodel/1/saved_model.pb (deflated 87%)
  adding: mm_spam_savedmodel/1/assets/ (stored 0%)
  adding: mm_spam_savedmodel/vocab.txt (deflated 47%)
